In [4]:
import os
import google.generativeai as genai
import pandas as pd
import time
import urllib

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.preview.generative_models as generative_models

#############################################
## GEMINI STUFF #############################
#############################################

def generate(input_text, safety_settings, generation_config):
    vertexai.init(project="mtrx-wg2-modeling-dev-9yj", location="us-east1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
    )
    responses = model.generate_content(
      [input_text],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
    )
    
    resText = ""
    for response in responses:
        resText+=response.text
        
    return resText

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}
safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
    ),
]

#############################################
## END OF GEMINI STUFF ######################
#############################################



# ORDER OF OPERATIONS FOR THIS SCRIPT
# 1. READ IN XLSX SHEET
# 2. MAKE DRUG NAMES UPPER AND PUT IN LIST
# 3. GET LIST OF INDICATIONS
# 4a. GENERATE DISEASES FROM INDICATIONS USING LLM
# 4b. Generate XLSX of rows consisting of drug name and list of indications interpreted from LLM.
# 5. FOR EACH ITEM IN DRUG LIST SET:
# 5.1 GET ALL INDICATIONS ASSOCIATED WITH ITEM, COLLECT.
# 5.2 APPEND [DrugName, Collection] to new dataframe
# 6. NAME RESOLVE TO GET ONTOLOGICAL MAPPING

In [5]:
pmda = pd.read_csv("pmda_approvals.csv")
drugnames = list(pmda['Active Ingredient (underlined: new active ingredient)'])
for idx, name in enumerate(drugnames):
    try:
        drugnames[idx] = name.upper()
    except:
        drugnames[idx] = drugnames[idx]

print(len(drugnames), 'rows of drugs')
print(len(set(drugnames)), 'unique drugs')


2448 rows of drugs
1296 unique drugs


In [7]:
indications = pmda['Indications']

print(len(indications), ' indications for human use')

from google.generativeai.types import HarmCategory, HarmBlockThreshold
import math
########################################
## WARNING: THIS COSTS MONEY TO RUN ####
##   THIS PORTION CONFIRMS YOUR RUN ####
########################################
print("WARNING: This section of the script uses Gemini resources - confirm that you want to run it?")
print("Waiting 5 seconds so you don't just hit continue without thinking...")
time.sleep(5)

successfulInput = False
while not successfulInput:
    scriptRun = input('Run the script? (yes / no)')
    if scriptRun == "yes":
        successfulInput = True
    elif scriptRun == "no":
        assert(0)
    else:
        print("Bad input. Please input ''yes'' / ''no''")
        successfulInput = False


############################################
## THIS PART ACTUALLY EXECUTES THE INGEST ##
############################################

indicationDiseaseList = []
drugsList = []
originalText = []

for index, ind in enumerate(indications):
    print(index)
    if not ind or type(ind)==float:
        print("found nan value")
        indicationDiseaseList.append("NA")
        drugsList.append(drugnames[index])
        originalText.append("NA")
    else:
        drugsList.append(drugnames[index])
        originalText.append(ind)
        try:
            prompt = "Produce a list of diseases treated in the following therapeutic indications list: " + ind +".\n Please format the list as ['item1', 'item2', ... ,'itemN']. Do not inlude any other text in the response. If no diseases are treated, return an empty list as '[]'. If the therapy is preventative, add the tag (preventative) to the item. If the drug is only used for diagnostic purposes, return 'diagnostic/contrast/radiolabel'."
            response = generate(prompt, safety_settings, generation_config)
            print(response)
            indicationDiseaseList.append(response.upper())
        except:
            print("LLM extraction failure")
            indicationDiseaseList.append("LLM failed to extract indications")

sheetData = pd.DataFrame(data=[drugsList,indicationDiseaseList, originalText]).transpose()
sheetData.columns = ['drug active ingredients', 'diseases', 'original text']
print(sheetData)
sheetData.to_excel("drug-disease-pairs-pmda.xlsx")

2448  indications for human use
Waiting 5 seconds so you don't just hit continue without thinking...


Run the script? (yes / no) yes


0


/Users/mdeluca/miniconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
I0000 00:00:1725043943.002991 2383968 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


['ulcerative colitis (excluding severe cases)'] 

1


...

['fever (preventative)', 'pain'] 

2433


I0000 00:00:1725045355.161320 2383968 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


['fever', 'pain (preventative)'] 

2434


I0000 00:00:1725045355.588602 2383968 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


['fever (preventative)', 'pain (preventative)'] 

2435
found nan value
2436
found nan value
2437
found nan value
2438
found nan value
2439
found nan value
2440
found nan value
2441
found nan value
2442
found nan value
2443
found nan value
2444
found nan value
2445
found nan value
2446
found nan value
2447
found nan value
     drug active ingredients  \
0                 BUDESONIDE   
1                  DANICOPAN   
2                CROVALIMAB    
3       BELUMOSUDIL MESILATE   
4          INCLISIRAN SODIUM   
...                      ...   
2443                     NaN   
2444                     NaN   
2445                     NaN   
2446                     NaN   
2447                     NaN   

                                               diseases  \
0     ['ULCERATIVE COLITIS (EXCLUDING SEVERE CASES)'...   
1           ['PAROXYSMAL NOCTURNAL HAEMOGLOBINURIA'] \n   
2           ['PAROXYSMAL NOCTURNAL HAEMOGLOBINURIA'] \n   
3     ['CHRONIC GRAFT-VERSUS-HOST DISEASE (PREVENTAT... 